In [1]:
!pip uninstall -y language_tool_python
!pip install language_tool_python


Found existing installation: language-tool-python 2.8
Uninstalling language-tool-python-2.8:
  Successfully uninstalled language-tool-python-2.8
  Using cached language_tool_python-2.8-py3-none-any.whl.metadata (12 kB)
Using cached language_tool_python-2.8-py3-none-any.whl (35 kB)


In [2]:
import os
import language_tool_python
from spellchecker import SpellChecker
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import spacy
from textblob import TextBlob
import dask.dataframe as dd
from dask.distributed import Client, progress
from dask import delayed, compute
from spellchecker import SpellChecker
import spacy
from textblob import TextBlob
import language_tool_python
import dask.dataframe as dd

In [3]:
import os
import gdown

def download_file_if_not_exists(file_url, output_path):
    """Скачивает файл с Google Drive, если он ещё не существует в указанной директории."""
    # Проверка наличия файла
    if os.path.exists(output_path):
        print(f"Файл '{output_path}' уже существует.")
    else:
        print(f"Файл '{output_path}' не найден. Начинаю загрузку...")
        gdown.download(file_url, output_path, quiet=False)
        print(f"Файл '{output_path}' успешно загружен.")

# Указываем URL и путь к файлу
file_url = 'https://drive.google.com/uc?id=15pofNbomaoUap41Rcn1uNGeiJIqFd2qe'
output_file_name = 'wildberries_reviews.csv.gz'  # Укажите реальное имя файла, которое хотите сохранить
output_path = os.path.join(os.getcwd(), output_file_name)  # Полный путь к файлу

download_file_if_not_exists(file_url, output_path)


Файл '/workspace/wildberries_reviews.csv.gz' не найден. Начинаю загрузку...


Downloading...
From (original): https://drive.google.com/uc?id=15pofNbomaoUap41Rcn1uNGeiJIqFd2qe
From (redirected): https://drive.google.com/uc?id=15pofNbomaoUap41Rcn1uNGeiJIqFd2qe&confirm=t&uuid=4d586577-9060-4c31-b74b-9995c6944c44
To: /workspace/wildberries_reviews.csv.gz
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 164M/164M [00:04<00:00, 40.6MB/s]

Файл '/workspace/wildberries_reviews.csv.gz' успешно загружен.


In [4]:
import subprocess

def install_spacy_model_if_not_exists(model_name):
    """Устанавливает модель spaCy, если она ещё не установлена."""
    try:
        # Получаем список установленных моделей
        result = subprocess.run(
            ['python', '-m', 'spacy', 'info'],
            capture_output=True, text=True, check=True
        )
        
        # Проверяем наличие модели в списке установленных
        if model_name in result.stdout:
            print(f"Модель {model_name} уже установлена.")
        else:
            print(f"Модель {model_name} не найдена. Устанавливаю...")
            subprocess.run(['python', '-m', 'spacy', 'download', model_name], check=True)
            print(f"Модель {model_name} успешно установлена.")
    
    except subprocess.CalledProcessError as e:
        print(f"Ошибка при выполнении команды: {e}")

# Замените 'ru_core_news_lg' на нужное имя модели
install_spacy_model_if_not_exists('ru_core_news_lg')

Модель ru_core_news_lg уже установлена.


In [5]:
!apt update && apt install -y default-jre wget unzip && wget https://languagetool.org/download/LanguageTool-stable.zip && unzip -o LanguageTool-stable.zip

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [861 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:6 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2112 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease               
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1421 kB]3m
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1130 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2393 kB]
Fetched 8175 kB in 1s (5704 kB/s)                      
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
10 package

In [6]:
import subprocess
# Запуск команды
process = subprocess.Popen(
    ['find', '/', '-name', 'languagetool-server.jar'],
    stdout=subprocess.PIPE,
    stderr=subprocess.DEVNULL,
    text=True
)

# Чтение вывода
output, _ = process.communicate()

# Удаление пробелов и пустых строк
languagetool_jar_path = [line.strip() for line in output.splitlines() if line.strip()][0]

In [7]:
import cudf.pandas  # Для автоматического ускорения pandas
import pandas as pd
import subprocess
import time
from language_tool_python import LanguageTool
import threading
import queue
from tqdm import tqdm
import os
import requests

cudf.pandas.install()  # Активирует режим ускорения pandas с использованием cuDF

# Параметры серверов
START_PORT = 8010
NUM_SERVERS = 20

# Хранение запущенных процессов и их портов
server_processes = {}

# Путь к файлу languagetool-server.jar
languagetool_jar_path = '/path/to/languagetool-server.jar'  # Убедитесь, что путь корректный

# Функция для запуска серверов LanguageTool
def start_languagetool_servers(start_port, num_ports):
    for port in range(start_port, start_port + num_ports):
        cmd = [
            'java',
            '-cp', languagetool_jar_path,
            'org.languagetool.server.HTTPServer',
            '--port', str(port),
            '--log-file', f'languagetool_server_{port}.log'  # Сохранение журнала
        ]
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        server_processes[port] = process
        print(f"Запущен сервер LanguageTool на порту {port}")
        
        # Динамическое ожидание и проверка доступности сервера
        max_wait_time = 60  # Максимальное время ожидания в секундах
        start_time = time.time()
        while time.time() - start_time < max_wait_time:
            if check_server(f'http://localhost:{port}'):
                break
            time.sleep(1)
        else:
            print(f"Не удалось дождаться запуска сервера на порту {port} в течение {max_wait_time} секунд.")

# Функция для проверки доступности сервера
def check_server(server_url):
    try:
        response = requests.post(f'{server_url}/v2/check', data={'text': 'test'}, headers={'Content-Type': 'application/x-www-form-urlencoded'})
        if response.status_code == 200:
            print(f"Сервер {server_url} доступен.")
            return True
        else:
            print(f"Сервер {server_url} вернул статус {response.status_code}.")
    except requests.RequestException as e:
        print(f"Ошибка подключения к серверу {server_url}: {e}")
    return False

# Функция для инициализации LanguageTool
def init_tool(server_url):
    if check_server(server_url):
        print(f"Инициализирую LanguageTool на сервере {server_url}...")
        try:
            return LanguageTool('ru-RU', remote_server=server_url)
        except Exception as e:
            print(f"Ошибка подключения к серверу {server_url}: {e}")
    return None

# Функция для исправления текста с использованием LanguageTool
def correct_text(text, tool):
    if not isinstance(text, str):
        return text
    try:
        corrected_text = tool.correct(text)
        # Если исправленный текст совпадает с исходным, возвращаем исходный текст
        return corrected_text if corrected_text != text else text
    except Exception as e:
        print(f"Ошибка исправления текста: {e}")
        return text

# Функция для обработки текста в потоке
def process_text(queue, results, progress_bar, batch_size):
    while not queue.empty():
        batch = []
        for _ in range(batch_size):
            if queue.empty():
                break
            idx, text, tool = queue.get()
            corrected_text = correct_text(text, tool)
            batch.append((idx, corrected_text))
            queue.task_done()
            # Обновляем прогресс-бар после исправления текста
            progress_bar.update(1)
        
        if batch:
            results.extend(batch)

# Функция для обработки данных
def process_data(file_path, num_rows, num_servers):
    # Инициализация серверов LanguageTool, если не все серверы уже запущены
    if len(server_processes) < num_servers:
        start_languagetool_servers(START_PORT + len(server_processes), num_servers - len(server_processes))

    # Список серверов
    languagetool_servers = [f'http://localhost:{port}' for port in server_processes.keys()]

    # Создание пула инструментов
    tools = [init_tool(url) for url in languagetool_servers]
    tools = [tool for tool in tools if tool is not None]  # Фильтрация неудачных инициализаций

    if not tools:
        print("Не удалось инициализировать LanguageTool.")
        return

    # Чтение данных с использованием cudf.pandas
    df = pd.read_csv(file_path, compression='gzip', nrows=num_rows)

    # Проверка и добавление колонки corrected_text, если ее нет
    if 'corrected_text' not in df.columns:
        df['corrected_text'] = ''

    # Инициализация очереди
    text_queue = queue.Queue()
    batch_size = 1000
    results = []

    # Заполнение очереди только строк без исправленного текста
    for idx, row in df.iterrows():
        if pd.isna(row['corrected_text']) or row['corrected_text'] == '':
            tool = tools[idx % len(tools)]  # Выбор инструмента по кругу
            text_queue.put((idx, row['review_full_text'], tool))

    # Инициализация прогресс-бара
    with tqdm(total=len(df)) as progress_bar:
        # Запуск потоков
        threads = []
        for _ in range(num_servers):
            thread = threading.Thread(target=process_text, args=(text_queue, results, progress_bar, batch_size))
            thread.start()
            threads.append(thread)

        # Ожидание завершения всех потоков
        text_queue.join()
        for thread in threads:
            thread.join()

    # Обновление DataFrame с исправленным текстом
    for idx, corrected_text in results:
        df.at[idx, 'corrected_text'] = corrected_text

    # Сохранение обновленного DataFrame обратно в исходный файл
    df.to_csv(file_path, compression='gzip', index=False)
    print(f"Обработка завершена для {num_rows} строк и {num_servers} серверов.")
    print(f"Результаты сохранены в {file_path}")

# Запуск обработки для разного количества серверов и строк
test_cases = [(1000, 1)]
for num_rows, num_servers in test_cases:
    process_data('wildberries_reviews.csv.gz', num_rows=num_rows, num_servers=num_servers)

# Завершение работы всех серверов
for port, process in server_processes.items():
    process.terminate()
    process.wait()
    print(f"Сервер на порту {port} завершил работу.")


Запущен сервер LanguageTool на порту 8010
Ошибка подключения к серверу http://localhost:8010: HTTPConnectionPool(host='localhost', port=8010): Max retries exceeded with url: /v2/check (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f6814633d90>: Failed to establish a new connection: [Errno 111] Connection refused'))
Ошибка подключения к серверу http://localhost:8010: HTTPConnectionPool(host='localhost', port=8010): Max retries exceeded with url: /v2/check (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f681444c610>: Failed to establish a new connection: [Errno 111] Connection refused'))
Ошибка подключения к серверу http://localhost:8010: HTTPConnectionPool(host='localhost', port=8010): Max retries exceeded with url: /v2/check (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f6814633bb0>: Failed to establish a new connection: [Errno 111] Connection refused'))
Ошибка подключения к серверу http://l

In [8]:
t_df = pd.read_csv('wildberries_reviews.csv.gz', nrows=110, compression='gzip')
t_df[(t_df["review_full_text"] != t_df["corrected_text"])]

KeyError: 'corrected_text'